<a href="https://colab.research.google.com/github/jeffersonramelo/Textual-Analysis/blob/main/C2_sentimento_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
# Caminho do arquivo CSV
caminho_arquivo = "/content/C123_dataset sentimento rotulados.csv"

# Ler o arquivo CSV
import pandas as pd
df = pd.read_csv(caminho_arquivo, sep=",", encoding='latin-1')

In [32]:
#vizualizar dados
df.head()

,sentimento_rotulado,texto,id
0,neutral,"According to Gran , the company has no plans t...",1
1,neutral,Technopolis plans to develop in stages an area...,2
2,negative,The international electronic industry company ...,3
3,positive,With the new production plant the company woul...,4
4,positive,According to the company 's updated strategy f...,5


In [33]:
#quantos dados positivo e negativos?
df.sentimento_rotulado.value_counts()

neutral     2879
positive    1363
negative     604
Name: sentimento_rotulado, dtype: int64

In [34]:
# Acessar os textos contidos nas celulas, só substituir o número da celula que deseja vizualizar
celula = df.loc[2, 'texto']

# Imprimir o conteúdo da célula
print(celula)


The international electronic industry company Elcoteq has laid off tens of employees from its Tallinn facility ; contrary to earlier layoffs the company contracted the ranks of its office workers , the daily Postimees reported .


pre-processamento do texto (stop words, misnusculas, caracteres especiais, stemização e tokenização)

In [36]:
pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [37]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stopwords_en = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
stopwords_en

In [39]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

def preprocess_text(text):
    # Converter para minúsculas
    text = text.lower()
    
    # Remover números
    text = re.sub(r'\d+', '', text)
    
    # Remover sinais de pontuação e caracteres especiais
    text = re.sub(r'[^\w\s]', '', text)
    
    # Remover stopwords
    stopwords_en = stopwords.words('english')
    text = ' '.join([word for word in text.split() if word not in stopwords_en])
    
    return text

df['texto'] = df['texto'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [40]:
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

# Criar uma instância do stemmer
stemmer = PorterStemmer()

# Função para realizar a stemização das palavras em um texto
def stem_text(text):
    # Tokenizar o texto em palavras
    words = word_tokenize(text)
    
    # Aplicar a stemização em cada palavra
    stemmed_words = [stemmer.stem(word) for word in words]
    
    # Juntar as palavras novamente em um texto
    stemmed_text = ' '.join(stemmed_words)
    
    return stemmed_text

# Aplicar a função de stemização na coluna "texto" do DataFrame
df['texto'] = df['texto'].apply(stem_text)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [41]:
df

,sentimento_rotulado,texto,id
0,neutral,accord gran compani plan move product russia a...,1
1,neutral,technopoli plan develop stage area less squar ...,2
2,negative,intern electron industri compani elcoteq laid ...,3
3,positive,new product plant compani would increas capac ...,4
4,positive,accord compani updat strategi year baswar targ...,5
...,...,...,...
4841,negative,london marketwatch share price end lower londo...,4842
4842,neutral,rinkuskiai beer sale fell per cent million lit...,4843
4843,negative,oper profit fell eur mn eur mn includ vessel s...,4844
4844,negative,net sale paper segment decreas eur mn second q...,4845


Naive Bayes

In [52]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.metrics import confusion_matrix, accuracy_score
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

# Dividir o DataFrame em dados de treinamento e teste
X = df['texto']
y = df['sentimento_rotulado']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preencher valores nulos com uma string vazia
imputer = SimpleImputer(strategy='constant', fill_value='')
X_train = pd.DataFrame(X_train, columns=['texto'])
X_test = pd.DataFrame(X_test, columns=['texto'])
X_train['texto'] = imputer.fit_transform(X_train[['texto']])
X_test['texto'] = imputer.transform(X_test[['texto']])

# Pré-processamento dos dados de texto
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train['texto'])
X_test = vectorizer.transform(X_test['texto'])

# Inicializar os classificadores
mnb = MultinomialNB()
gnb = GaussianNB()
bnb = BernoulliNB()

# Realizar validação cruzada nos classificadores
cv_scores_mnb = cross_val_score(mnb, X_train, y_train, cv=5)
cv_scores_gnb = cross_val_score(gnb, X_train.toarray(), y_train, cv=5)
cv_scores_bnb = cross_val_score(bnb, X_train, y_train, cv=5)

# Treinar os classificadores nos dados de treinamento completos
mnb.fit(X_train, y_train)
gnb.fit(X_train.toarray(), y_train)
bnb.fit(X_train, y_train)

# Fazer previsões nos dados de teste
y_pred_mnb = mnb.predict(X_test)
y_pred_gnb = gnb.predict(X_test.toarray())
y_pred_bnb = bnb.predict(X_test)


In [46]:
# Calcular a matriz de confusão
confusion_mnb = confusion_matrix(y_test, y_pred_mnb)
print("confusion_mnb", confusion_mnb)
confusion_gnb = confusion_matrix(y_test, y_pred_gnb)
print("confusion_gnb", confusion_gnb)
confusion_bnb = confusion_matrix(y_test, y_pred_bnb)
print("confusion_bnb", confusion_bnb)

confusion_mnb [[ 93  57  29]
 [ 33 723  91]
 [ 25 177 226]]
confusion_gnb [[100  46  33]
 [104 445 298]
 [ 98 154 176]]
confusion_bnb [[ 19  99  61]
 [  0 786  61]
 [  3 214 211]]


In [53]:
# Avaliar o desempenho dos classificadores com a métrica de acurácia
accuracy_mnb = accuracy_score(y_test, y_pred_mnb)
print("accuracy_mnb", accuracy_mnb)
accuracy_gnb = accuracy_score(y_test, y_pred_gnb)
print("accuracy_gnb", accuracy_gnb)
accuracy_bnb = accuracy_score(y_test, y_pred_bnb)
print("accuracy_bnb", accuracy_bnb)

accuracy_mnb 0.7164948453608248
accuracy_gnb 0.49381443298969074
accuracy_bnb 0.7164948453608248


Suporte Vector Machine

In [54]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score

# Dividir o DataFrame em dados de treinamento e teste
X = df['texto']
y = df['sentimento_rotulado']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pré-processamento dos dados de texto
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Inicializar o classificador SVM
svm = SVC()

# Treinar o classificador SVM
svm.fit(X_train, y_train)

# Fazer previsões nos dados de teste
y_pred_svm = svm.predict(X_test)

# Gerar a matriz de confusão
confusion_mtx_svm = confusion_matrix(y_test, y_pred_svm)

print("Matriz de Confusão - SVM:")
print(confusion_mtx_svm)

# Calcular a acurácia do modelo SVM
accuracy_svm = accuracy_score(y_test, y_pred_svm)

print("Acurácia - SVM:", accuracy_svm)


Matriz de Confusão - SVM:
[[ 39  56  15]
 [  3 557  11]
 [ 14 169 106]]
Acurácia - SVM: 0.7237113402061855


Redes Neurais

A rede neural utilizada no exemplo é um Perceptron de Múltiplas Camadas (Multilayer Perceptron, MLP). O MLP é uma arquitetura de rede neural artificial que consiste em várias camadas de neurônios, incluindo uma camada de entrada, uma ou mais camadas ocultas e uma camada de saída. Cada neurônio em uma camada está conectado a todos os neurônios na camada seguinte, formando uma rede densamente conectada.

In [56]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score

# Dividir o DataFrame em dados de treinamento e teste
X = df['texto']
y = df['sentimento_rotulado']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pré-processamento dos dados de texto
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Inicializar o classificador MLP
mlp = MLPClassifier(hidden_layer_sizes=(100,), random_state=42)

# Treinar o classificador MLP
mlp.fit(X_train, y_train)

# Fazer previsões nos dados de teste
y_pred_mlp = mlp.predict(X_test)

# Gerar a matriz de confusão
confusion_mtx_mlp = confusion_matrix(y_test, y_pred_mlp)

print("Matriz de Confusão - MLP:")
print(confusion_mtx_mlp)

# Calcular a acurácia do modelo MLP
accuracy_mlp = accuracy_score(y_test, y_pred_mlp)

print("Acurácia - MLP:", accuracy_mlp)


Matriz de Confusão - MLP:
[[ 61  30  19]
 [ 27 460  84]
 [ 16 109 164]]
Acurácia - MLP: 0.7061855670103093
